In [1]:
import pandas as pd
from googleapiclient.discovery import build
import warnings
warnings.filterwarnings('ignore')

In [2]:
mv = pd.read_csv('data/kpop_mv_with_video_seconds.csv')

In [3]:
api_key = 'please use your own key ^^'
youtube = build('youtube', 'v3', developerKey=api_key)

In [5]:
mv.head()

,Date,Artist,Song Name,Korean Name,Director,Video,Type,Release,video_seconds
0,2020-05-22,Agust D,Daechwita,대취타,NaN,https://youtu.be/qGjAWJ2zWWI,Boy Solo,Major,269
1,2020-05-21,Yubin,yaya (Me Time),넵넵,NaN,https://youtu.be/BJD0arHF_5c,Girl Solo,Major,186
2,2020-05-21,OnlyOneOf,Angel (prod. GRAY),NaN,NaN,https://youtu.be/vVTo8p72FxQ,Boy,Major,221
3,2020-05-20,Ryu Sujeong,Tiger Eyes,NaN,NaN,https://youtu.be/aE6curPGQRY,Girl Solo,Major,225
4,2020-05-20,Crush,Mayday feat. Joy (Red Velvet),자나깨나,NaN,https://youtu.be/29ycT6fA-Rs,Boy Solo,Major,201


In [6]:
mv['real_id'] = 0
mv['viewCount'] = 0
mv['likeCount'] = 0
mv['commentCount'] = 0
for i in range(len(mv)):
    vid = mv.Video[i][17:]
    request = youtube.videos().list(part = 'statistics', id = vid)
    response = request.execute()

    if not response['items']:
        continue
    else:
        mv.real_id[i] = vid
        mv.viewCount[i] = int(response['items'][0]['statistics']['viewCount']) if ('viewCount' in response['items'][0]['statistics'].keys()) else 0
        mv.likeCount[i] = int(response['items'][0]['statistics']['likeCount']) if ('likeCount' in response['items'][0]['statistics'].keys()) else 0
        mv.commentCount[i] = int(response['items'][0]['statistics']['commentCount']) if ('commentCount' in response['items'][0]['statistics'].keys()) else 0

In [13]:
for i in range(len(mv)):
    if mv.real_id[i] == '0':
        search_keyword = str(mv.Artist[i]) + ' ' + str(mv['Song Name'][i]) 
        get_id_request = youtube.search().list(q=search_keyword, part='snippet')
        res = get_id_request.execute()
        target_vid = res['items'][0]['id']['videoId']
        
        request = youtube.videos().list(part = 'statistics', id = target_vid)
        response = request.execute()

        mv.real_id[i] = target_vid
        mv.viewCount[i] = int(response['items'][0]['statistics']['viewCount']) if ('viewCount' in response['items'][0]['statistics'].keys()) else 0
        mv.likeCount[i] = int(response['items'][0]['statistics']['likeCount']) if ('likeCount' in response['items'][0]['statistics'].keys()) else 0
        mv.commentCount[i] = int(response['items'][0]['statistics']['commentCount']) if ('commentCount' in response['items'][0]['statistics'].keys()) else 0

deal with duplicates

In [17]:
mv.head()

,Date,Artist,Song Name,Korean Name,Director,Video,Type,Release,video_seconds,real_id,viewCount,likeCount,commentCount
0,2020-05-22,Agust D,Daechwita,대취타,NaN,https://youtu.be/qGjAWJ2zWWI,Boy Solo,Major,269,qGjAWJ2zWWI,400055012,14169804,1361748
1,2020-05-21,Yubin,yaya (Me Time),넵넵,NaN,https://youtu.be/BJD0arHF_5c,Girl Solo,Major,186,BJD0arHF_5c,1164316,53308,2757
2,2020-05-21,OnlyOneOf,Angel (prod. GRAY),NaN,NaN,https://youtu.be/vVTo8p72FxQ,Boy,Major,221,vVTo8p72FxQ,7492213,81482,3994
3,2020-05-20,Ryu Sujeong,Tiger Eyes,NaN,NaN,https://youtu.be/aE6curPGQRY,Girl Solo,Major,225,aE6curPGQRY,2645267,42965,3526
4,2020-05-20,Crush,Mayday feat. Joy (Red Velvet),자나깨나,NaN,https://youtu.be/29ycT6fA-Rs,Boy Solo,Major,201,29ycT6fA-Rs,16993641,652327,32200


In [28]:
duplicate_list = mv.real_id.value_counts().reset_index()
duplicate_list.columns = ['real_id', 'cnt']

In [30]:
len(duplicate_list[duplicate_list.cnt != 1])

7

In [31]:
len(mv)

3772

In [32]:
len(mv.drop_duplicates(subset='real_id', keep='last'))

3765

In [33]:
mv = mv.drop_duplicates(subset='real_id', keep='last')
mv.to_csv('data/kpop_mv_with_all_info.csv', index=False)